In [2]:
import subprocess

cmd = (
    'rsync -avz --progress /mnt/vol1/rsync_test wsl:/mnt/d/project'
)

with subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as proc:
    for line in proc.stdout:
        print(line, end='')
    proc.wait()

if proc.returncode != 0:
    raise subprocess.CalledProcessError(proc.returncode, cmd)


sending incremental file list
rsync_test/hel.txt

             25 100%    0.00kB/s    0:00:00  
             25 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=0/2)

sent 171 bytes  received 42 bytes  142.00 bytes/sec
total size is 25  speedup is 0.12


In [19]:
import subprocess

cmd = (
    'rsync -avz --progress /mnt/vol1/rsync_test ubuntu:/media/me/ssdsata/project/'
)

with subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as proc:
    print(proc.stdout)
    for line in proc.stdout:
        print(line, end='')
    proc.wait()

if proc.returncode != 0:
    raise subprocess.CalledProcessError(proc.returncode, cmd)

<_io.TextIOWrapper name=79 encoding='UTF-8'>
sending incremental file list
rsync_test/counter.txt

            700  14%    0.00kB/s    0:00:00  
          4,788 100%    3.90MB/s    0:00:00 (xfr#1, to-chk=1/3)

sent 298 bytes  received 81 bytes  151.60 bytes/sec
total size is 4,819  speedup is 12.72


In [16]:
from typing import Optional
import subprocess

def ping_host(ip: str, count: int = 3, timeout: int = 2) -> tuple[bool, Optional[str]]:
    """Ping a host and return success status and latency"""
    try:
        result = subprocess.run(
            ["ping", "-c", str(count), "-W", str(timeout), ip],
            capture_output=True,
            text=True,
            timeout=timeout * count + 2
        )
        # print(result)
        if result.returncode == 0:
            # Extract average latency from ping output
            for line in result.stdout.split('\n'):
                # print(line)
                if 'avg' in line or 'min/avg/max' in line:
                    # print(line)
                    parts = line.split('/')
                    print(parts)
                    if len(parts) >= 5:
                        avg_latency = parts[4]
                        return True, f"{avg_latency}ms"
            return True, "success"
        return False, "timeout"
    except Exception as e:
        return False, str(e)

In [10]:
ubuntu_ip = '192.168.193.40'
wsl_ip = '192.168.193.10'

ping_host(ubuntu_ip)

CompletedProcess(args=['ping', '-c', '3', '-W', '2', '192.168.193.40'], returncode=0, stdout='PING 192.168.193.40 (192.168.193.40) 56(84) bytes of data.\n64 bytes from 192.168.193.40: icmp_seq=1 ttl=64 time=230 ms\n64 bytes from 192.168.193.40: icmp_seq=2 ttl=64 time=137 ms\n64 bytes from 192.168.193.40: icmp_seq=3 ttl=64 time=159 ms\n\n--- 192.168.193.40 ping statistics ---\n3 packets transmitted, 3 received, 0% packet loss, time 2002ms\nrtt min/avg/max/mdev = 137.295/175.392/229.606/39.371 ms\n', stderr='')
PING 192.168.193.40 (192.168.193.40) 56(84) bytes of data.
64 bytes from 192.168.193.40: icmp_seq=1 ttl=64 time=230 ms
64 bytes from 192.168.193.40: icmp_seq=2 ttl=64 time=137 ms
64 bytes from 192.168.193.40: icmp_seq=3 ttl=64 time=159 ms

--- 192.168.193.40 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2002ms
rtt min/avg/max/mdev = 137.295/175.392/229.606/39.371 ms


(True, '175.392ms')

In [15]:
ping_host(ubuntu_ip)

rtt min/avg/max/mdev = 80.692/102.250/124.583/17.926 ms


(True, '102.250ms')

In [17]:
ping_host(ubuntu_ip)

['rtt min', 'avg', 'max', 'mdev = 119.613', '142.585', '167.735', '19.705 ms']


(True, '142.585ms')

In [11]:
ping_host(wsl_ip)

CompletedProcess(args=['ping', '-c', '3', '-W', '2', '192.168.193.10'], returncode=1, stdout='PING 192.168.193.10 (192.168.193.10) 56(84) bytes of data.\nFrom 192.168.193.147 icmp_seq=1 Destination Host Unreachable\nFrom 192.168.193.147 icmp_seq=2 Destination Host Unreachable\nFrom 192.168.193.147 icmp_seq=3 Destination Host Unreachable\n\n--- 192.168.193.10 ping statistics ---\n3 packets transmitted, 0 received, +3 errors, 100% packet loss, time 2052ms\npipe 3\n', stderr='')


(False, 'timeout')